# Zadanie zaliczeniowe NYPD

In [1]:
#znaczna część pokryta testami

### Zadanie:
Zadaniem zaliczeniowym jest **stworzenie biblioteki do przetwarzania pewnego typu danych**.<br>
Do biblioteki powinien być załączony skrypt (może być w formie Jupyter notebooka) pokazujący użycie biblioteki na konkretnych danych.<br>
Własne propozycje zadań również muszą to spełniać.

## Podstawowa propozycja zadania:

W tym roku prosimy Państwa o zbadanie zależności między:
 - dochodami państw (GDP - gross domestic product, PKB - produkt krajowy brutto),
 - liczbami mieszkańców,
 - i emisją substancji lotnych.

## Dane:


- **GDP** (plik csv, jest w spakowanym pliku zip).<br>Tu są dane o GDP dla wszystkich krajów w latach 1960-2021 (niektórych danych brakuje).
- **Liczby mieszkańców** (plik csv, jest w spakowanym plik zip). Dane o ludności w krajach też z lat 1960-2021.
- **Emisja CO2** (plik csv, jest też obok spakowany plik z danymi w formacie csv, i json w środku). Dane o emisji z lat 1751 (!) do 2014. Na stronie są też przykładowe fragmenty kodu pobierającego te dane

### [To robi argparse]

In [37]:
import pandas as pd
import argparse

In [2]:
#populacja
population_path = "./data/API_SP/API_SP.POP.TOTL_DS2_en_csv_v2_4751604.csv"
#GDP
gdp_path = "./data/API_NY/API_NY.GDP.MKTP.CD_DS2_en_csv_v2_4751562.csv"
#CO2
co2_path = "./data/co2-fossil-by-nation_zip/data/fossil-fuel-co2-emissions-by-nation_csv.csv"

## Polecenie
Należy napisać w Pythonie program, który:
1. [ ] <span style="color:blue">Pobiera (używając argparse) z wiersza poleceń ścieżki do plików (w formacie csv) z danymi o GDP, populacji i emisji. **[później]**</span><br><br>
2. [x] Zakłada że są w formacie jaki dziś (choć w przyszłości danych może być więcej, czyli nie można założyć, że program ma działać dokładnie na tych plikach, które są dołączone do zadania, natomiast należy założyć, że format będzie zachowany) i je wczytuje.<br><br>
3. [x] Czyści te dane (w tym punkcie nie ma wiele do zrobienia w odniesieniu do podanych źródeł danych).<br><br>
4. [x] Wybiera tylko te lata, które są we wszystkich tabelach. Uwaga: chodzi o lata faktycznie występujące w tych tabelach, także w przyszłości, tzn. nie można w docelowej wersji na sztywno założyć (zaszyć w kodzie), że są to dane za lata 1960-2014), choć w pierwszych wersjach rozwiązania to może być wygodne.
5. [x] Scala dane po krajach i latach.<br><br>
6. [ ] Przeprowadza analizy (za pomocą bibliotek numpy lub panadas):<br>
    - **Które kraje w poszczególnych latach z danymi, emitują najwięcej CO2 w przeliczeniu na mieszkańca.**<br>To znaczy generuje posortowaną po latach tabelkę pięcioma krajami o największej emisji na osobę (z podaną nazwą kraju, emisją na osobę i całkowitą emisją.
    - **Które kraje w poszczególnych latach z danymi mają największy przychód mieszkańca.**<br>To znaczy generuje posortowaną po latach tabelkę pięcioma krajami o największym dochodzie na mieszkańca (z podaną nazwą kraju, dochodem na mieszkańca i całkowitym dochodem).
    - **Które kraje (w przeliczeniu na mieszkańca) najbardziej zmniejszyły i zwiększyły przez ost. 10 lat (z danych) emisję CO2.**

# część I<br>PRZYGOTOWANIE DANYCH

## Wczytanie danych

In [3]:
df_population_raw = pd.read_csv(population_path, skiprows=4)
df_gdp_raw = pd.read_csv(gdp_path, skiprows=4)
df_co2_raw = pd.read_csv(co2_path)

## 1. Przygotowanie tabel z danymi
#### <span style="color:red">sprowadzić wartości liczbowe do wspólnych typów danych <span>

### 1.1 Przygotowanie tabeli z danymi z worldbank:

Funkcja przygotowująca z danych wczytanych z pliku w formacie worldbank ramkę z danymi w formacie lata x kraje. 

In [4]:
def prepare_worldbank_df(raw_df):
    '''Funkcja przygotowuje dane (GDP/populacja) w formacie jak w 'data.worldbank.org'
    do analizy (tabela postaci lata x kraje)'''
    
    df = raw_df.T
    
    #czy niepowtarzalne nazwy w wierszu 'Country name'
    assert len(set(df.loc['Country Name'])) == len(df.columns)
    
    #ustaw nagłówek
    countries_caps = [c.upper() for c in df.loc['Country Name']]
    df.set_axis(list(countries_caps), axis=1, inplace=True)
    
    #pozostawiamy tylko wiersze z latami (w tym formacie tabeli: od czwartego poza nagłówkiem do przedostatniego)
    df = df[4:-1]
    
    return df
    

In [5]:
df_population= prepare_worldbank_df(df_population_raw)
df_gdp = prepare_worldbank_df(df_gdp_raw)

In [7]:
#print(df_gdp['Afghanistan'][0])
#type(df_gdp['Afghanistan'][0])

### 1.2 Przygotowanie tabeli z danymi o emisji CO2:

In [6]:
def co2_to_year_vs_country_format(co2_df):
    '''Funkcja konwertuje ramkę danych, zawierającą rekordy o emisji CO2 w danym roku przez dany kraj
    (ramka musi zawierać kolumny: 'Year', 'Country', 'Total')
    do ramki, w której indeksami są wartości z 'Year', nazwami kolumn wartości z 'Country', a wartościami zawartość kolumny'Total' '''
    
    ## w oryginalnych danych kolumna 'Year' przechowuje typ 'int', w worlbank to 'str' 
    df = co2_df.astype({'Year': 'str'})
    return df.pivot(index='Year', columns='Country', values='Total')

In [7]:
df_co2 = co2_to_year_vs_country_format(df_co2_raw)

## 2. Wstępne filtrowanie po wspólnych nazwach krajów i latach

In [8]:
def crop_to_shared_labels(dfs:list, axis=0)->list:
    '''Funkcja dla zadanej listy ramek danych, zwraca ramki przefiltrowane po wspólnych etykietach
    (indeksach (axis=0) lub nazwach kolumn (axis=1))'''
    
    assert len(dfs) > 0
    assert axis in {0,1}
    
    ax = 'index' if not axis else 'columns'
    
    shared_labels = set(dfs[0].eval(ax))
    for df in dfs[1:]:
        shared_labels.intersection_update(set(df.eval(ax)))
    shared_labels = sorted(list(shared_labels))
    
    dfs_filtered_sorted = [df.filter(items=shared_labels, axis=axis) for df in dfs]
    return dfs_filtered_sorted

In [9]:
def print_nrows_ncols(dfs:list):
    for i,df in enumerate(dfs):
        print(f"({i}) wiersze: {len(df.index)}, kolumny: {len(df.columns)}")

In [11]:
dfs = [df_population, df_gdp, df_co2]
dfs = [df.astype(float) for df in dfs]

print_nrows_ncols(dfs)

(0) wiersze: 62, kolumny: 266
(1) wiersze: 62, kolumny: 266
(2) wiersze: 264, kolumny: 256


In [12]:
dfs = [df_population, df_gdp, df_co2]
dfs = crop_to_shared_labels(dfs, axis=0)
dfs = crop_to_shared_labels(dfs, axis=1)
#df_population, df_gdp, df_co2 = dfs

print_nrows_ncols(dfs)

(0) wiersze: 55, kolumny: 161
(1) wiersze: 55, kolumny: 161
(2) wiersze: 55, kolumny: 161


## 3. Usuwanie wartości NaN

### 3.1 Usuwanie kolumn (krajów) zawierających wartości NaN

In [14]:
def remove_nan_columns(df):
    #axis: 0->remove rows, 1->remove columns
    return df.dropna(axis=1)

In [15]:
#usuwanie kolumn z NaN
dfs = [remove_nan_columns(df) for df in dfs]

print_nrows_ncols(dfs)

(0) wiersze: 55, kolumny: 159
(1) wiersze: 55, kolumny: 75
(2) wiersze: 55, kolumny: 113


### 3.2 Ponowne ujedolicenie wspólnych krajów i lat

In [16]:
dfs = crop_to_shared_labels(dfs)
dfs = crop_to_shared_labels(dfs, axis=1)

print_nrows_ncols(dfs)

(0) wiersze: 55, kolumny: 62
(1) wiersze: 55, kolumny: 62
(2) wiersze: 55, kolumny: 62


### 3.3 Sortowanie ramek po latach i krajach:

In [19]:
def sort_dfs(dfs):
    dfs_sorted = [df.sort_index(axis=0) for df in dfs]
    dfs_sorted = [df.sort_index(axis=1) for df in dfs_sorted]
    return dfs_sorted

In [20]:
dfs = sort_dfs(dfs)
#print(dfs[0].index==dfs[2].index)

# część II<br>ANALIZY

6. [ ] Przeprowadza analizy (za pomocą bibliotek numpy lub panadas):<br>
    - **Które kraje w poszczególnych latach z danymi, emitują najwięcej CO2 w przeliczeniu na mieszkańca.**<br>To znaczy generuje posortowaną po latach tabelkę pięcioma krajami o największej emisji na osobę (z podaną nazwą kraju, emisją na osobę i całkowitą emisją.
    - **Które kraje w poszczególnych latach z danymi mają największy przychód mieszkańca.**<br>To znaczy generuje posortowaną po latach tabelkę pięcioma krajami o największym dochodzie na mieszkańca (z podaną nazwą kraju, dochodem na mieszkańca i całkowitym dochodem).
    - **Które kraje (w przeliczeniu na mieszkańca) najbardziej zmniejszyły i zwiększyły przez ost. 10 lat (z danych) emisję CO2.**

In [21]:
df_population, df_gdp, df_co2 = dfs

In [22]:
#utworzenie ramek danych do analiz 1 i 2
df_co2_per_capita = (df_co2 / df_population).astype(float).sort_index(axis=0)
df_gdp_per_capita = (df_gdp / df_population).astype(float).sort_index(axis=0)
df_co2_per_capita.head()
#df_gdp_per_capita.head()

Country,ALGERIA,AUSTRALIA,AUSTRIA,BELGIUM,BELIZE,BENIN,BERMUDA,BRAZIL,BURKINA FASO,CANADA,...,SRI LANKA,SURINAME,SWEDEN,SYRIAN ARAB REPUBLIC,THAILAND,TOGO,TRINIDAD AND TOBAGO,UGANDA,UNITED KINGDOM,URUGUAY
Year,,,,,,,,,,,,,,,,,,,,,
1960,0.000152,0.002341,0.001193,0.002711,0.000130,0.000018,0.000968,0.000177,0.000002,0.002937,...,0.000062,0.000410,0.001793,0.000192,0.000037,0.000011,0.000830,0.000017,0.003041,0.000464
1961,0.000146,0.002357,0.001226,0.002755,0.000106,0.000014,0.001055,0.000181,0.000005,0.002898,...,0.000063,0.000395,0.001772,0.000177,0.000040,0.000016,0.001451,0.000016,0.003042,0.000437
1962,0.000133,0.002410,0.001297,0.002902,0.000195,0.000015,0.000923,0.000191,0.000005,0.003035,...,0.000068,0.000444,0.001851,0.000179,0.000047,0.000015,0.002245,0.000016,0.003039,0.000420
1963,0.000124,0.002516,0.001406,0.003105,0.000170,0.000013,0.000860,0.000193,0.000005,0.003036,...,0.000065,0.000465,0.001986,0.000196,0.000051,0.000018,0.000400,0.000016,0.003069,0.000447
1964,0.000126,0.002661,0.001470,0.003014,0.000223,0.000015,0.001125,0.000191,0.000006,0.003356,...,0.000057,0.000515,0.002151,0.000229,0.000066,0.000022,0.001147,0.000017,0.003072,0.000466


### (A) Które kraje w poszczególnych latach z danymi, emitują najwięcej CO2 w przeliczeniu na mieszkańca.
To znaczy generuje posortowaną po latach tabelkę pięcioma krajami o największej emisji na osobę (z podaną nazwą kraju, emisją na osobę i całkowitą emisją.

In [23]:
def top_5_scores(df_per_capita, df_total, k=5):
    '''Funkcja dla przekazanych ramek danych - z wartościami per capita i z wartościami sumarycznie,
    zwraca ramkę danych, posortowaną po latach, w z krajami o top k wartościach w ramce per-capita
    i wartościach dla tych krajów w ramce df_total.
    Uwaga - obie rami muzą mieć zgodne indeksy oraz nazwy kolumn.'''
    
    assert (df_per_capita.index == df_total.index).all()
    assert (df_per_capita.columns == df_total.columns).all()
    
    df_list = []
    
    for year in df_per_capita.index:
        largest_5 = df_per_capita.loc[year].nlargest(k)
        largest_5_totals = df_total.loc[year][largest_5.index]
        
        df_c = pd.concat([largest_5, largest_5_totals], axis=1).reset_index()
        df_c.insert(0,'Year', year)
        df_c.columns = ['Year', 'Country', 'Per capita', 'Total']
        df_list.append(df_c)
    df_all = pd.concat(df_list, axis=0) #, ignore_index=True)
    #df_all = df_all.sort_values('Year')
    
    return df_all

In [24]:
top_5_scores(df_co2_per_capita, df_co2)

,Year,Country,Per capita,Total
0,1960,LUXEMBOURG,0.010004,3141.0
1,1960,UNITED KINGDOM,0.003041,159340.0
2,1960,CANADA,0.002937,52603.0
3,1960,BELGIUM,0.002711,24816.0
4,1960,AUSTRALIA,0.002341,24053.0
...,...,...,...,...
0,2014,TRINIDAD AND TOBAGO,0.009263,12619.0
1,2014,LUXEMBOURG,0.004735,2634.0
2,2014,AUSTRALIA,0.004197,98517.0
3,2014,CANADA,0.004134,146494.0


### (B) Które kraje w poszczególnych latach z danymi mają największy przychód mieszkańca.
To znaczy generuje posortowaną po latach tabelkę pięcioma krajami o największym dochodzie na mieszkańca (z podaną nazwą kraju, dochodem na mieszkańca i całkowitym dochodem).

In [25]:
top_5_scores(df_gdp_per_capita, df_gdp)

,Year,Country,Per capita,Total
0,1960,CANADA,2259.294285,40461721692.646797
1,1960,LUXEMBOURG,2242.015817,703925705.942958
2,1960,SWEDEN,2114.002973,15822585033.576401
3,1960,BERMUDA,1902.402119,84466654.076912
4,1960,AUSTRALIA,1810.619230,18606786874.23
...,...,...,...,...
0,2014,LUXEMBOURG,123678.702143,68804811897.644501
1,2014,BERMUDA,98467.683994,6413988000.0
2,2014,NORWAY,97019.182753,498410050251.255981
3,2014,AUSTRALIA,62511.690590,1467504819608.919922


### (C) Które kraje (w przeliczeniu na mieszkańca) najbardziej zmniejszyły i zwiększyły przez ost. 10 lat (z danych) emisję CO2.

In [28]:
#df_co2 - posortowane po indeksach
df_co2_per_capita
df_co2_per_capita.iloc[-10].name
len(df_co2_per_capita) <10 

def top_differences(df, k=10):
    '''Funkcja dla podanej ramki danych wypisuje nazwy kolumn (krajów),
    dla których odnotowano najwyższy wzrost/spadek w wartościach między ostatnim a -k-tym wierszem (rokiem).
    Jeżeli dla żadnej z kolumn nie nastąpił wzrost/(spadek), wypisany zostanie odpowiedni komunikat.
    Uwaga: funkcja nie sortuje danych, uzytkownik zapewnia dane w wybranym przez siebie porządku.'''
    
    assert k >0
    if len(df) < k:
        print(f"Zadany przedział lat ({k}) większy niż liczba dostępnych danych ({len(df)} lat).\Podaj mniejszy przedział")
        return
    
    diffs = df.iloc[-1] - df.iloc[-k]
    ctr_max, ctr_min = diffs.idxmax(), diffs.idxmin()
    
    #kraj o największej zmianie w emisji między ostatnim wierszem w danych a k-tym
    if diffs[ctr_max] > 0:
        print(f"Kraj o największym wzroście emisji CO2 na osobę w przeciągu {k} ostatnich lat:")
        print(f"{ctr_max}: wzrost o {round(diffs[ctr_max], 6)} na osobę")
    else:
        print("Żaden z uwzględnionych krajów nie zwiększył emisji względem początku rozpatrywanego przedziału.")
    
    #największy spadek w emisji
    if diffs[ctr_min] < 0:
        print(f"\nKraj o największym spadku emisji CO2 na osobę w przeciągu {k} ostatnich lat:")
        print(f"{ctr_min}: spadek o {round(-diffs[ctr_min], 6)} na osobę")
    else:
        print("\nŻaden z uwzględnionych krajów nie zwiększył emisji względem początku rozpatrywanego przedziału.")
    

In [29]:
top_differences(df_co2_per_capita)

Kraj o największym wzroście emisji CO2 na osobę w przeciągu 10 ostatnich lat:
TRINIDAD AND TOBAGO: wzrost o 0.001238 na osobę

Kraj o największym spadku emisji CO2 na osobę w przeciągu 10 ostatnich lat:
LUXEMBOURG: spadek o 0.002035 na osobę


# część III<br>Dodatki techniczne
- podział na moduły
- zrobić testy
- zrobić profilowanie kodu
- gotowe -- do instalacji